In [1]:
# Source of the DS: https://archive.ics.uci.edu/dataset/577/codon+usage

In [46]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from imblearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter

from sklearn.ensemble import IsolationForest

In [3]:
# Load the DS
df = pd.read_csv('datasets/codon_usage.csv')
df.head()

C:\Users\Home\AppData\Local\Temp\ipykernel_32652\2935273154.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/codon_usage.csv')


,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [4]:
df.info()
# that is a huge ds

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13028 entries, 0 to 13027
Data columns (total 69 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Kingdom      13028 non-null  object 
 1   DNAtype      13028 non-null  int64  
 2   SpeciesID    13028 non-null  int64  
 3   Ncodons      13028 non-null  int64  
 4   SpeciesName  13028 non-null  object 
 5   UUU          13028 non-null  object 
 6   UUC          13028 non-null  object 
 7   UUA          13028 non-null  float64
 8   UUG          13028 non-null  float64
 9   CUU          13028 non-null  float64
 10  CUC          13028 non-null  float64
 11  CUA          13028 non-null  float64
 12  CUG          13028 non-null  float64
 13  AUU          13028 non-null  float64
 14  AUC          13028 non-null  float64
 15  AUA          13028 non-null  float64
 16  AUG          13028 non-null  float64
 17  GUU          13028 non-null  float64
 18  GUC          13028 non-null  float64
 19  GUA 

In [5]:
df.columns

Index(['Kingdom', 'DNAtype', 'SpeciesID', 'Ncodons', 'SpeciesName', 'UUU',
       'UUC', 'UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA',
       'AUG', 'GUU', 'GUC', 'GUA', 'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU',
       'CCC', 'CCA', 'CCG', 'UGG', 'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC',
       'UCA', 'UCG', 'AGU', 'AGC', 'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC',
       'CAA', 'CAG', 'AAU', 'AAC', 'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG',
       'CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG',
       'UAA', 'UAG', 'UGA'],
      dtype='object')

In [6]:
# Let's check the duplicated rows
print(f'DS shape:',df.shape)
number_of_duplicated_rows =df.duplicated().sum()
print(f'Number of duplicated rows:{number_of_duplicated_rows}')
# I dont have duplicated rows

DS shape: (13028, 69)
Number of duplicated rows:0


In [7]:
df['UUU']=pd.to_numeric(df['UUU'],errors='coerce')
df['UUC'] = pd.to_numeric(df['UUC'],errors='coerce')

# The DS is big, let's check the unique values in each column.
for column in df.columns:
        print(f"Unique values in {column}: {df[column].unique()}")

Unique values in Kingdom: ['vrl' 'arc' 'bct' 'phg' 'plm' 'pln' 'inv' 'vrt' 'mam' 'rod' 'pri']
Unique values in DNAtype: [ 0  6  4  2  1  3  7  9  5 11 12]
Unique values in SpeciesID: [100217 100220 100755 ...   9601   9602   9606]
Unique values in Ncodons: [    1995     1474     4862 ...    96254 40662582  8998998]
Unique values in SpeciesName: ['Epizootic haematopoietic necrosis virus' 'Bohle iridovirus'
 'Sweet potato leaf curl virus' ...
 'mitochondrion Pongo pygmaeus pygmaeus' 'Homo sapiens'
 'mitochondrion Homo sapiens']
Unique values in UUU: [0.01654 0.02714 0.01974 ... 0.02314 0.01791 0.00554]
Unique values in UUC: [0.01203 0.01357 0.0218  ... 0.03789 0.03215 0.03555]
Unique values in UUA: [0.0005  0.00068 0.01357 ... 0.03671 0.02285 0.04059]
Unique values in UUG: [0.00351 0.00678 0.01543 ... 0.0004  0.00491 0.00619]
Unique values in CUU: [0.01203 0.00407 0.00782 ... 0.00504 0.0299  0.03415]
Unique values in CUC: [0.03208 0.02849 0.01111 ... 0.04792 0.05322 0.05042]
Unique value

In [8]:
# let's check do I have columns with only one unique value
unique_values = df.columns[df.nunique() == 1]
unique_values

Index([], dtype='object')

In [9]:
df["Kingdom"].value_counts()

Kingdom
bct    2920
vrl    2832
pln    2523
vrt    2077
inv    1345
mam     572
phg     220
rod     215
pri     180
arc     126
plm      18
Name: count, dtype: int64

In [10]:
# Kingdom colum shows biological groups,
# bct = bacteria, vrl = virus
# pln = plant
# inv = invertebrate, mam = mammal 
# To understand better those abbreviations, I used QWEN 3MAX.

In [11]:
#  I am going to drop those classes which have few samples.
df= df[~df["Kingdom"].isin(["arc", "inv", "mam", "phg", "plm",  "pri", "rod"])]
df.shape

(10352, 69)

In [12]:
df["Kingdom"].value_counts()

Kingdom
bct    2920
vrl    2832
pln    2523
vrt    2077
Name: count, dtype: int64

In [13]:
# The classes'name are ambiguous, I am going map them.
kingdom_map = {'bct':'Bacteria', 'vrl':'Virus', 'pln':'Plant', 'vrt':'Vertebrate'}
df['Kingdom'] = df['Kingdom'].map(kingdom_map)

In [14]:
df['DNAtype'].value_counts()

DNAtype
0    7737
1    1765
2     815
4      28
3       2
9       2
6       1
7       1
5       1
Name: count, dtype: int64

In [15]:
# Having ID number and Species name is cheat code for model.
df = df.drop(columns=['SpeciesID','SpeciesName', 'DNAtype'])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10352 entries, 0 to 12060
Data columns (total 66 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Kingdom  10352 non-null  object 
 1   Ncodons  10352 non-null  int64  
 2   UUU      10350 non-null  float64
 3   UUC      10351 non-null  float64
 4   UUA      10352 non-null  float64
 5   UUG      10352 non-null  float64
 6   CUU      10352 non-null  float64
 7   CUC      10352 non-null  float64
 8   CUA      10352 non-null  float64
 9   CUG      10352 non-null  float64
 10  AUU      10352 non-null  float64
 11  AUC      10352 non-null  float64
 12  AUA      10352 non-null  float64
 13  AUG      10352 non-null  float64
 14  GUU      10352 non-null  float64
 15  GUC      10352 non-null  float64
 16  GUA      10352 non-null  float64
 17  GUG      10352 non-null  float64
 18  GCU      10352 non-null  float64
 19  GCC      10352 non-null  float64
 20  GCA      10352 non-null  float64
 21  GCG      10352 no

In [17]:
df[['UUU','UUC']].isna().sum()

UUU    2
UUC    1
dtype: int64

In [18]:
# there are not many NaN values, I can drop 
df =df.dropna(subset=['UUU','UUC'])

In [19]:
# Now I have to convert Kingdom column to numerical values
class_encoder =LabelEncoder()
df['Kingdom']= class_encoder.fit_transform(df['Kingdom'])
df['Kingdom'].value_counts()

Kingdom
0    2919
3    2831
1    2523
2    2077
Name: count, dtype: int64

In [20]:
summary = df.describe().T
summary

,count,mean,std,min,25%,50%,75%,max
Kingdom,10350.0,1.465700,1.166219,0.0,0.000000,1.000000,3.000000,3.000000e+00
Ncodons,10350.0,80976.708599,593271.618032,1000.0,1576.000000,2904.000000,9768.500000,3.413228e+07
UUU,10350.0,0.023592,0.014709,0.0,0.013590,0.021820,0.031140,1.298300e-01
UUC,10350.0,0.022631,0.011124,0.0,0.015030,0.021345,0.028047,9.169000e-02
UUA,10350.0,0.019010,0.017759,0.0,0.005258,0.014940,0.028170,1.411300e-01
...,...,...,...,...,...,...,...,...
GAA,10350.0,0.028824,0.014110,0.0,0.018162,0.027100,0.037120,1.448900e-01
GAG,10350.0,0.022366,0.014445,0.0,0.011633,0.021305,0.031238,1.585500e-01
UAA,10350.0,0.001546,0.001441,0.0,0.000520,0.001300,0.002270,1.942000e-02
UAG,10350.0,0.000588,0.000848,0.0,0.000000,0.000430,0.000840,2.561000e-02


In [21]:
# Ncodons is skewed,  what I see from median and mean. 
# UAG, UAA, and UGA are very close to zero, when I asked GPT, the response was: 
# These are condon, which instruction for cell to adds one amino acid for building  protein. 
# When it is very low that means, STOP building the chain. 

In [22]:
print("class mapping:",dict(zip(class_encoder.classes_,range(len(class_encoder.classes_)))))
# classes are imbalanced, I will handle it later.

class mapping: {'Bacteria': 0, 'Plant': 1, 'Vertebrate': 2, 'Virus': 3}


In [23]:
# let's check can I find any useful information from those columns and kingdom column
stoped_condons = ['UAG', 'UAA', 'UGA']
for column in stoped_condons:
    grouped = df.groupby("Kingdom")[column].quantile([0.5, 0.95, 0.99]).unstack().round(4)
    print(f"\n{column}:")
    display(grouped)
# The gap between 0.5 and 0.95 is very big, but between 0.95 and 0.99 is not that big
# that means there are some outliers in the data, but not many.



UAG:


,0.50,0.95,0.99
Kingdom,,,
0,0.0005,0.0014,0.0021
1,0.0004,0.0018,0.0033
2,0.0004,0.0029,0.0043
3,0.0004,0.0018,0.0029



UAA:


,0.50,0.95,0.99
Kingdom,,,
0,0.0013,0.0032,0.0046
1,0.0010,0.0032,0.0049
2,0.0024,0.0070,0.0078
3,0.0009,0.0036,0.0046



UGA:


,0.50,0.95,0.99
Kingdom,,,
0,0.0011,0.0034,0.0080
1,0.0008,0.0028,0.0133
2,0.0242,0.0316,0.0370
3,0.0005,0.0022,0.0035


### Filteing the best columns 

In [24]:
# My aim was to merge redundant columns, 64 columns is too much for analysis.
X = df.drop(columns=['Kingdom'])
y = df['Kingdom']
print(X.shape, y.shape)
print(y.value_counts())

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

(10350, 65) (10350,)
Kingdom
0    2919
3    2831
1    2523
2    2077
Name: count, dtype: int64


In [25]:
# Selecting the best features for this ds is hard, this is a hard ds. 
# So I am going create pipeline from many methods and using cv.

# Spliting training data into stratified folds.
cv = StratifiedKFold(5, shuffle=True, random_state=42)

ks= [k for k in (10, 15, 20, 25, 30) if k <= x_train.shape[1]]

pipe=Pipeline([("ros", RandomOverSampler(random_state=42)), ("sel", SelectKBest(mutual_info_classif)),
               ("rf", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))])

search = GridSearchCV(pipe, {"sel__k": ks}, scoring="f1_macro", cv=cv, n_jobs=-1).fit(x_train, y_train)
print("k_best:", search.best_params_["sel__k"])
print("cv_macro_f1:", round(search.best_score_, 4))
print(x_train.columns[search.best_estimator_.named_steps["sel"].get_support()].tolist())

# For debugging, I used QWEN 3MAX.

k_best: 30
cv_macro_f1: 0.941
['Ncodons', 'UUG', 'CUU', 'CUC', 'CUA', 'AUC', 'AUA', 'GUG', 'GCC', 'GCG', 'CCA', 'CCG', 'UGG', 'GGU', 'GGC', 'UCU', 'ACC', 'ACA', 'CAG', 'UGU', 'AAA', 'AAG', 'CGU', 'CGC', 'AGA', 'AGG', 'GAU', 'GAA', 'GAG', 'UGA']


In [26]:
# Creating a new df based on selected features and target column.
df = x_train.loc[:, x_train.columns[search.best_estimator_.named_steps["sel"].get_support()]].assign(Kingdom=y_train.values)
df.shape

(7245, 31)

In [27]:
df.head()

,Ncodons,UUG,CUU,CUC,CUA,AUC,AUA,GUG,GCC,GCG,...,AAG,CGU,CGC,AGA,AGG,GAU,GAA,GAG,UGA,Kingdom
5958,21810,0.01453,0.02196,0.01389,0.01385,0.02054,0.01293,0.01087,0.01137,0.01013,...,0.02082,0.01137,0.00628,0.01032,0.00523,0.03586,0.03966,0.02577,0.00023,0
6339,1428,0.02311,0.02101,0.00000,0.02101,0.02031,0.00420,0.01050,0.01891,0.00840,...,0.00420,0.02731,0.00840,0.01050,0.00420,0.03782,0.05252,0.01471,0.00000,1
7209,1028,0.01946,0.00584,0.00292,0.00000,0.03016,0.00292,0.00681,0.01751,0.00097,...,0.01362,0.00681,0.00000,0.01362,0.00097,0.04280,0.04086,0.01654,0.00097,1
3684,6779,0.01814,0.01269,0.00693,0.00428,0.01608,0.00605,0.02095,0.01578,0.01888,...,0.02641,0.00944,0.00752,0.00561,0.00162,0.04617,0.03216,0.02449,0.00030,0
4293,963629,0.01906,0.01080,0.00322,0.00960,0.01468,0.00688,0.01466,0.01279,0.01405,...,0.01211,0.01328,0.00606,0.00607,0.00114,0.03664,0.06177,0.01263,0.00045,0


In [32]:
df.columns

Index(['Ncodons', 'UUG', 'CUU', 'CUC', 'CUA', 'AUC', 'AUA', 'GUG', 'GCC',
       'GCG', 'CCA', 'CCG', 'UGG', 'GGU', 'GGC', 'UCU', 'ACC', 'ACA', 'CAG',
       'UGU', 'AAA', 'AAG', 'CGU', 'CGC', 'AGA', 'AGG', 'GAU', 'GAA', 'GAG',
       'UGA', 'Kingdom'],
      dtype='object')

In [34]:
# Handling the imbalanced classes:
# I am aiming to create a pipeline to try different method like smote, random oversampling, ADASYN and compare them 
x = df.drop(columns=['Kingdom'])
y = df['Kingdom']
print(y.value_counts())
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.25, random_state=42, stratify=y)

Kingdom
0    2043
3    1982
1    1766
2    1454
Name: count, dtype: int64


In [35]:
X = df.drop(columns=["Kingdom"]).astype(float)
vif = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
print("VIF:", vif )
print(vif.sort_values(ascending=False).head(10).round(1))

VIF: Ncodons     1.036454
UUG         7.631089
CUU         8.225078
CUC         9.384572
CUA        11.290893
AUC        14.121818
AUA         6.687741
GUG        10.052428
GCC        16.282472
GCG        11.375568
CCA         8.233641
CCG        10.917677
UGG         8.170837
GGU         8.338945
GGC        15.828050
UCU         8.149001
ACC        13.849620
ACA         9.029703
CAG         9.155549
UGU         4.683440
AAA        12.200386
AAG        10.390357
CGU         5.760085
CGC        11.471107
AGA         8.807220
AGG         5.687761
GAU        17.095038
GAA        16.415642
GAG        12.955065
UGA         9.261443
dtype: float64
GAU    17.1
GAA    16.4
GCC    16.3
GGC    15.8
AUC    14.1
ACC    13.8
GAG    13.0
AAA    12.2
CGC    11.5
GCG    11.4
dtype: float64


In [36]:
high_vif = vif[vif > 10].sort_values(ascending=False).index.tolist()
high_vif

# I am gonna drop those columns with high VIF.
df = df.drop(columns=high_vif)
df.shape

(7245, 17)

In [44]:
# Check duplicated rows
df.duplicated().sum()
df = df.drop_duplicates()

In [57]:
features = df.columns.drop('Kingdom')
iso_mpdel = IsolationForest(contamination=0.05, random_state=42, n_jobs=-1).fit_predict(df[features]) == -1
print(f"Isolation forest outliers:{iso_mpdel.sum()}")

Isolation forest outliers:343


In [58]:
Q1 = df[features].quantile(0.25)
Q3 = df[features].quantile(0.75)
IQR = Q3 - Q1
iqr_outlier = ((df[features] < (Q1 - 1.5*IQR)) | (df[features] > (Q3 + 1.5*IQR))).any(axis=1)
print("IQR outliers:", iqr_outlier.sum())

IQR outliers: 2922


In [59]:
print(f"Both flagged:{(iqr_outlier & iso_mpdel).sum()}")
df = df[~(iqr_outlier & iso_mpdel)]
df.shape

Both flagged:338


(6509, 18)

In [60]:
from scipy import stats

In [61]:
# Find noisy values using z-score (|z| > 3) and replace with median
features = df.columns.drop('Kingdom')
z = np.abs(stats.zscore(df[features]))
noisy_mask = z > 3
print("Noisy values per column:")
print(noisy_mask.sum().to_string())

Noisy values per column:
Ncodons      56
UUG          24
CUU          93
CUC          78
AUA          62
CCA          38
UGG          46
GGU          57
UCU          91
ACA          27
CAG          37
UGU          97
CGU          46
AGA          22
AGG          39
UGA         270
iso_flag      0


In [62]:
# Replace noisy values with column median
for col in features:
    median_val = df[col].median()
    df.loc[noisy_mask[col], col] = median_val
print(f"Total noisy values replaced: {noisy_mask.sum().sum()}")
print(f"Shape unchanged: {df.shape}")

Total noisy values replaced: 1083
Shape unchanged: (6509, 18)


In [63]:
# Check class overlap: compare class means relative to pooled std
class_means = df.groupby('Kingdom')[features].mean()
pooled_std = df[features].std()
# Range of class means / pooled std = separability score
sep = (class_means.max() - class_means.min()) / pooled_std
print("Separability score per feature (higher = less overlap):")
print(sep.sort_values().round(3).to_string())

Separability score per feature (higher = less overlap):
Ncodons     0.662
CAG         0.902
CUU         1.093
AUA         1.144
GGU         1.167
UCU         1.203
UGG         1.298
CUC         1.312
CGU         1.340
ACA         1.374
AGG         1.396
UGU         1.410
CCA         1.435
UUG         1.491
AGA         1.541
UGA         1.697
iso_flag      NaN


In [64]:
df.shape

(6509, 18)

In [ ]:
# Handling the imbalanced classes based on macro f1, which means all classes are important equally.
# The highest average f1 score across all classes is the best.
# I am gonna use 3 different methods: RandomOverSampler, SMOTE, BorderlineSMOTE 

# Training with  original data
# RandomOverSampler, this method copies the miniority 
# smote, creating new samples of minority 
# Border, that is similar to smote but it focuses on borderline samples.

oversampling_setup ={"Base":None,"ROS":RandomOverSampler(random_state=42),  
                    "SMOTE":SMOTE(random_state=42),"BORDER":BorderlineSMOTE(random_state=42)}



best_f1_average= ("", float("-inf"), float("-inf"))
for name, method in oversampling_setup.items():
    # First, I am going to use the original data without oversampling.
    if method is None:
        x_train_oversampled, y_train_oversampled = x_train, y_train
    else :
        x_train_oversampled, y_train_oversampled= method.fit_resample(x_train, y_train)

    print(f"\n{name} train counts after:",Counter(y_train_oversampled))
    model = RandomForestClassifier(n_estimators=500, random_state=42,n_jobs=-1).fit(x_train_oversampled, y_train_oversampled)
    predicted_class = model.predict(x_test)
    # Calculating the f1 score for each class 
    f_score_per_class =f1_score(y_test, predicted_class, average=None)
    macro= f1_score(y_test, predicted_class, average="macro")
    smallest_value =float(f_score_per_class.min())

    print("macro_f1:",round(macro,4),"per_class_f1:",[round(score,4) for score in f_score_per_class])
    if (macro>best_f1_average[1])or(macro==best_f1_average[1]and smallest_value>best_f1_average[2]):best_f1_average=(name, macro, smallest_value)

print(f"\nBest F1 average: {best_f1_average[0]}. macro={best_f1_average[1]:.4f}, min_class_f1={best_f1_average[2]:.4f}")
# ROS model has the best macro f1 score.
# For debuuging the tuple initialization with sentinel values and best score tracking, I used QWEN 3MAX.


Base train counts after: Counter({0: 2189, 3: 2123, 1: 1892, 2: 1558})
macro_f1: 0.964 per_class_f1: [0.9705, 0.9449, 0.9864, 0.954]

ROS train counts after: Counter({3: 2189, 1: 2189, 2: 2189, 0: 2189})
macro_f1: 0.9639 per_class_f1: [0.9705, 0.9448, 0.9854, 0.9548]

SMOTE train counts after: Counter({3: 2189, 1: 2189, 2: 2189, 0: 2189})
macro_f1: 0.9638 per_class_f1: [0.9698, 0.944, 0.9844, 0.9572]

BORDER train counts after: Counter({3: 2189, 1: 2189, 2: 2189, 0: 2189})
macro_f1: 0.9647 per_class_f1: [0.9719, 0.9445, 0.9864, 0.9559]

Best F1 average: BORDER. macro=0.9647, min_class_f1=0.9445
